In [55]:
import requests, json, os
import folium
import geopandas as gpd
from shapely.geometry import shape
from pyproj import CRS


In [56]:
# Get data API
# Tambahkan &limit=10000 untuk membuka data API secara keseluruhan

import requests

url = "https://geoserver.mapid.io/layers_new/get_layer?api_key=d96fcd98a91f4954ba5a51a8de2c3554&layer_id=6890de6d623dcfb726e90c33&project_id=67d241e182926d98678eea8b&limit=10000"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("Successfully Get API")
else:
    print("Failed to Get API", response.status_code)

Successfully Get API


In [57]:
# Visualisasi Get API

import folium
data = requests.get(url).json()

m = folium.Map(location = [0, 0], zoom_start = 2)

gj = folium.GeoJson(data, name="geojson").add_to(m)

m.fit_bounds(gj.get_bounds())

m


In [58]:
# Melihat  daftar kolom dan tipe data yang tersedia

data = requests.get(url).json()

features = data.get("features", [])

jumlah_baris = len(features)

jumlah_kolom = len(features[0].get("properties",{})) if features else 0

print("Jumlah Baris (features):", jumlah_baris)
print("Jumlah Kolom (properties):", jumlah_kolom)

if not features:
    print("tidak ada fitur dalam data")
else:
    sample_props = features[0].get("properties", {})

    print("===Struktur Kolom & Tipe Data===")

    for key, value in sample_props.items():
        print(f"{key} -> {type(value).__name__}")

Jumlah Baris (features): 151
Jumlah Kolom (properties): 6
===Struktur Kolom & Tipe Data===
fid -> int
KABKOT -> str
KECAMATAN -> str
DESA -> str
JUMLAH PENDUDUK -> int
KELAS -> str


In [59]:
# Melakukan Pengecekan CRS

import requests, json

url = "https://geoserver.mapid.io/layers_new/get_layer?api_key=d96fcd98a91f4954ba5a51a8de2c3554&layer_id=6890de6d623dcfb726e90c33&project_id=67d241e182926d98678eea8b&limit=10000"

data = requests.get(url).json()

feats = data.get("features", [])

print("CRS:", data.get("crs"))

CRS: None


In [ ]:
# Melakukan Pengecekan CRS melalui sample data koordinat

from pyproj import CRS

feats = data.get("features", [])
if not feats:
    print("Features Not Found")
else:
    coords = feats [0].get("geometry", {}).get("coordinates", [])
    if coords and isinstance(coords[0], list):
        sample = coords[0][0][:5]
        print("sample coordinate:", sample)

        lon, lat = sample[0][:2]

        if -180 <= lon <= 180 and -90 <= lat <= 90:
            crs = CRS.from_epsg(4326)
            print("CRS: EPSG 4326 (WGS 84)")
        else:
            crs = CRS.from_epsg(3857)
            print("Cannot Read Coordinates")

sample coordinate: [[107.66286008600008, -6.946295783999972, 0], [107.66271072000006, -6.947030355999971, 0], [107.66270222300005, -6.947072140999978, 0], [107.66268838800005, -6.947229967999931, 0], [107.66268583300007, -6.947259107999969, 0]]
CRS: EPSG 4326 (WGS 84)


In [62]:
# Melakukan Manipulasi Data API

# Get data API
# Tambahkan &limit=10000 untuk membuka data API secara keseluruhan

import requests

url = "https://geoserver.mapid.io/layers_new/get_layer?api_key=d96fcd98a91f4954ba5a51a8de2c3554&layer_id=6890de6d623dcfb726e90c33&project_id=67d241e182926d98678eea8b&limit=10000"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("Successfully Get API")
else:
    print("Failed to Get API", response.status_code)


Successfully Get API


In [63]:
# Melakukan Pengecekan Data Awal

features = data.get("features", [])
if features:
    sample_props = features[0].get("properties", {})
    print("List of Column", list(sample_props.keys()))
else:
    print("Column Not Found")

List of Column ['fid', 'KABKOT', 'KECAMATAN', 'DESA', 'JUMLAH PENDUDUK', 'KELAS']


In [65]:
# Melakukan Penambahan Kolom Baru

from shapely.geometry import shape

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    for i, feature in enumerate(data.get('features', [])):
        props = feature.get("properties", {})
        kelas = str(props.get("KELAS", "")).upper()
        props["SKOR"] = {"RENDAH": 3, "SEDANG": 2, "TINGGI": 1}.get(kelas,0)

        geom = shape(feature.get("geometry", {}))
        luas_km2 = geom.area / 1e6

        props["LUAS KM2"] = float(luas_km2)

        if i <5:
            print(props)

            features = data.get("features", [])
            if features:
                cols = list(features[0]["properties"].keys())
                print("\nDaftar Kolom Setelah Ditambah Kolom SKOR dan LUAS KM2")
            else:
                print ("Failed, status_code: {response.status_code}")

{'fid': 69356, 'KABKOT': 'KOTA BANDUNG', 'KECAMATAN': 'BUAHBATU', 'DESA': 'SEKEJATI', 'JUMLAH PENDUDUK': 26161, 'KELAS': 'TINGGI', 'SKOR': 1, 'LUAS KM2': 1.7538486452844158e-10}

Daftar Kolom Setelah Ditambah Kolom SKOR dan LUAS KM2
{'fid': 69354, 'KABKOT': 'KOTA BANDUNG', 'KECAMATAN': 'GEDEBAGE', 'DESA': 'RANCABOLANG', 'JUMLAH PENDUDUK': 12528, 'KELAS': 'SEDANG', 'SKOR': 2, 'LUAS KM2': 2.1449914653332513e-10}

Daftar Kolom Setelah Ditambah Kolom SKOR dan LUAS KM2
{'fid': 69353, 'KABKOT': 'KOTA BANDUNG', 'KECAMATAN': 'REGOL', 'DESA': 'PASIRLUYU', 'JUMLAH PENDUDUK': 17541, 'KELAS': 'TINGGI', 'SKOR': 1, 'LUAS KM2': 8.422720888624037e-11}

Daftar Kolom Setelah Ditambah Kolom SKOR dan LUAS KM2
{'fid': 69350, 'KABKOT': 'KOTA BANDUNG', 'KECAMATAN': 'BABAKAN CIPARAY', 'DESA': 'MARGASUKA', 'JUMLAH PENDUDUK': 12755, 'KELAS': 'SEDANG', 'SKOR': 2, 'LUAS KM2': 9.64911842390299e-11}

Daftar Kolom Setelah Ditambah Kolom SKOR dan LUAS KM2
{'fid': 69349, 'KABKOT': 'KOTA BANDUNG', 'KECAMATAN': 'BUAHBAT

In [67]:
# Melakukan pengecekan tipe data di data terbaru

if features:
    tipe_kolom = {}
    sample = features[0]["properties"]

    for k, v in sample.items():
        tipe_kolom[k] = type(v).__name__

    print("\nTipe data dalam kolom")
    for k, t in tipe_kolom.items():
            print(f"-{k}:{t}")
else:
     print("Features Not Found")


Tipe data dalam kolom
-fid:int
-KABKOT:str
-KECAMATAN:str
-DESA:str
-JUMLAH PENDUDUK:int
-KELAS:str
-SKOR:int
-LUAS KM2:float


In [37]:
# Export data GeoJson

import json, os

out = r"C:\Users\Rizky\Videos\WebGIS\WebGIS Python\Hasil_Proses_Python.geojson"

os.makedirs(os.path.dirname(out), exist_ok=True)

with open(out, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

    print("File Successfully Saved to:", out)

File Successfully Saved to: C:\Users\Rizky\Videos\WebGIS\WebGIS Python\Hasil_Proses_Python.geojson
